In [2]:
import pandas as pd
import random

### Write function to create dummy data

In [3]:
def create_dummy_data(list_of_colnames, row_num):

    # Create list of list where each sublist is a row containing 1s & 0s
    rows = [[random.randint(0, 1) for i in range(len(list_of_colnames))] for i in range(row_num)]
    # Using nested list of rows and list of column names create dataframe
    dataframe = pd.DataFrame(rows, columns=list_of_colnames)

    # Add userId column relevant for Biobank data
    dataframe.insert(0, 'userId', list(range(row_num)))

    return dataframe

### Write function to turn dataframe to tsv

In [4]:
def df_to_tsv(dataframe, filename):

    dataframe.to_csv(filename, sep='\t', index=False)

    return f'{filename} created.'

### Create actual dummy disease data

In [5]:
# Create list of column headers
columns = [f'disease_{i}' for i in range(10)]

# Create dataframe with 50,000 rows
df = create_dummy_data(columns, 50000)

### Write dummy data to tsv file

In [6]:
# df_to_tsv(df, '~/Desktop/GitHub/ThyCa_Multimorbidity_UKBB/data_exploration/disease_dummy_data.tsv')